

---

# **Chapter 50: Package Ecosystem Guide**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Identify and evaluate essential Flutter and Dart packages for production applications
- Assess package quality using pub.dev scoring metrics and community indicators
- Manage version constraints effectively using semantic versioning and dependency resolution
- Create reusable Flutter packages with proper structure and documentation
- Publish packages to pub.dev following industry best practices and maintenance guidelines
- Handle dependency conflicts and transitive dependency management
- Implement private package hosting for enterprise development

---

## **Prerequisites**

- Completed Part I-IV (Flutter and Dart fundamentals)
- Understanding of object-oriented programming and generics
- Familiarity with command-line tools and Git version control
- Basic knowledge of software licensing and open-source compliance

---

## **50.1 Understanding the Pub Package Manager**

Flutter and Dart use the `pub` package manager to handle dependencies. Understanding how pub works is essential for managing professional Flutter projects.

### **The pubspec.yaml File**

The `pubspec.yaml` file is the heart of your Flutter project's configuration. It defines metadata, dependencies, and project structure.

```yaml
# Project metadata
name: my_flutter_app
description: A comprehensive Flutter application demonstrating industry standards.
# The description appears on pub.dev if you publish the package
# It should be 60-180 characters for optimal display

version: 1.2.3+4
# Format: MAJOR.MINOR.PATCH+BUILD
# MAJOR: Breaking changes
# MINOR: New features (backward compatible)
# PATCH: Bug fixes
# BUILD: Android version code / iOS CFBundleVersion

publish_to: 'none'
# Use 'none' to prevent accidental publishing to pub.dev
# Remove or set to specific URL for private pub servers

environment:
  sdk: '>=3.0.0 <4.0.0'
  # Constrains the Dart SDK version
  # 3.0.0 is the minimum (null safety required)
  # <4.0.0 ensures compatibility with Dart 3.x
  
  flutter: ">=3.10.0"
  # Optional: Constrains Flutter SDK version

dependencies:
  flutter:
    sdk: flutter
  # The Flutter SDK dependency is required for all Flutter apps
  
  # Production dependencies (required for app to run)
  cupertino_icons: ^1.0.2
  # ^ allows updates that don't modify the left-most non-zero digit
  # ^1.0.2 allows >=1.0.2 <2.0.0
  
  dio: 5.3.0
  # No prefix means exact version only (not recommended for apps)
  
  bloc: ">=8.0.0 <9.0.0"
  # Range constraint: any version between 8.0.0 and 9.0.0
  
  riverpod: 
    hosted:
      name: riverpod
      url: https://pub.mycompany.com
    version: ^2.0.0
  # Private hosted package example

dev_dependencies:
  # Dependencies only needed for development/testing
  flutter_test:
    sdk: flutter
  build_runner: ^2.4.0
  json_serializable: ^6.7.0

dependency_overrides:
  # Temporary overrides to force specific versions
  # Useful for resolving conflicts during development
  http: ^1.0.0

flutter:
  # Flutter-specific configuration
  uses-material-design: true
  
  assets:
    - assets/images/
    - assets/fonts/
    
  fonts:
    - family: CustomFont
      fonts:
        - asset: assets/fonts/CustomFont-Regular.ttf
        - asset: assets/fonts/CustomFont-Bold.ttf
          weight: 700
```

**Explanation:**

- **`name`**: Must be lowercase with underscores (snake_case). This is your package identifier.
- **`version`**: Follows semantic versioning (semver). The `+4` is the build number, used by Android as `versionCode` and iOS as `CFBundleVersion`.
- **`environment`**: Constrains the SDK versions your app supports. Dart 3.0.0+ enforces null safety.
- **`dependencies`**: Packages required for your app to run in production.
- **`dev_dependencies`**: Packages only needed during development (testing, code generation, linting). These aren't included in release builds.
- **Version constraints**:
  - `^1.0.2` (caret): Allows updates that don't change the left-most non-zero digit. `^1.0.2` allows `1.0.3`, `1.1.0`, but not `2.0.0`.
  - `>=1.0.0 <2.0.0` (range): Explicit range specification.
  - `1.0.0` (exact): Pins to exact version (fragile, avoids updates).
- **`dependency_overrides`**: Forces specific package versions, bypassing normal resolution. Useful for hotfixes but should be temporary.

### **Pub Commands and Workflow**

Understanding pub commands is essential for dependency management.

```bash
# Install dependencies defined in pubspec.yaml
flutter pub get
# This creates/updates:
# - pubspec.lock (locks exact versions)
# - .dart_tool/package_config.json (resolution metadata)

# Upgrade dependencies to latest allowed by constraints
flutter pub upgrade
# Updates pubspec.lock with newest compatible versions
# Does NOT update pubspec.yaml constraints

# Upgrade specific package
flutter pub upgrade dio

# View outdated packages
flutter pub outdated
# Shows current | resolvable | latest versions
# Helps identify when to widen constraints

# Analyze project dependencies
flutter pub deps
# Shows dependency tree (transitive dependencies)
# Use --style=tree for visual hierarchy

# Publish package (for package authors)
flutter pub publish --dry-run
# --dry-run validates without publishing
# Requires proper LICENSE, README, and pubspec.yaml

# Login to pub.dev
flutter pub login
# Authenticates for publishing
# Stores credentials in ~/.pub-cache/credentials.json

# Cache management
flutter pub cache repair
# Reinstalls all packages from cache (fixes corruption)
flutter pub cache clean
# Clears global cache (use with caution)
```

**Explanation:**

- **`flutter pub get`**: Reads `pubspec.yaml`, resolves dependencies (choosing versions that satisfy all constraints), downloads packages to `~/.pub-cache/hosted/pub.dev/`, and generates `pubspec.lock`.
- **`pubspec.lock`**: The lock file records exact versions used. Commit this for applications to ensure reproducible builds. For libraries (packages), don't commit it.
- **Transitive dependencies**: When you add Package A which depends on Package B, pub automatically installs both. `flutter pub deps` shows this tree.
- **Version solving**: Pub uses constraint satisfaction algorithms to find compatible versions. If constraints conflict (e.g., Package A needs `http: ^0.13.0`, Package B needs `http: ^1.0.0`), pub fails with a resolution error.

---

## **50.2 Essential Package Categories**

Industry-standard Flutter applications rely on curated packages. Here are the essential categories with recommended packages.

### **Networking and HTTP**

```dart
import 'package:dio/dio.dart';
import 'package:retrofit/retrofit.dart';
import 'package:json_annotation/json_annotation.dart';

part 'api_service.g.dart';

// Data model with JSON serialization
@JsonSerializable()
class User {
  final String id;
  final String name;
  final String email;
  
  User({required this.id, required this.name, required this.email});
  
  // Generated code handles JSON conversion
  factory User.fromJson(Map<String, dynamic> json) => _$UserFromJson(json);
  Map<String, dynamic> toJson() => _$UserToJson(this);
}

// REST API interface using Retrofit (type-safe HTTP client)
@RestApi(baseUrl: "https://api.example.com")
abstract class ApiService {
  factory ApiService(Dio dio, {String baseUrl}) = _ApiService;
  
  @GET("/users")
  Future<List<User>> getUsers();
  
  @GET("/users/{id}")
  Future<User> getUser(@Path("id") String id);
  
  @POST("/users")
  Future<User> createUser(@Body() User user);
  
  @PUT("/users/{id}")
  Future<User> updateUser(@Path("id") String id, @Body() User user);
  
  @DELETE("/users/{id}")
  Future<void> deleteUser(@Path("id") String id);
}

// Dio configuration with interceptors
class DioClient {
  static Dio getInstance() {
    final dio = Dio(BaseOptions(
      baseUrl: 'https://api.example.com',
      connectTimeout: Duration(seconds: 30),
      receiveTimeout: Duration(seconds: 30),
      headers: {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
      },
    ));
    
    // Add logging interceptor for debugging
    dio.interceptors.add(LogInterceptor(
      request: true,
      requestHeader: true,
      requestBody: true,
      responseHeader: true,
      responseBody: true,
      error: true,
    ));
    
    // Add authentication interceptor
    dio.interceptors.add(InterceptorsWrapper(
      onRequest: (options, handler) async {
        // Add auth token to every request
        final token = await _getAuthToken();
        options.headers['Authorization'] = 'Bearer $token';
        handler.next(options);
      },
      onError: (error, handler) async {
        // Handle 401 Unauthorized globally
        if (error.response?.statusCode == 401) {
          await _refreshToken();
          // Retry request with new token
          handler.resolve(await _retry(error.requestOptions));
        } else {
          handler.next(error);
        }
      },
    ));
    
    return dio;
  }
  
  static Future<String> _getAuthToken() async {
    // Implementation using flutter_secure_storage
    return 'token';
  }
  
  static Future<void> _refreshToken() async {
    // Token refresh logic
  }
  
  static Future<Response<dynamic>> _retry(RequestOptions requestOptions) async {
    final dio = getInstance();
    return dio.request(
      requestOptions.path,
      data: requestOptions.data,
      queryParameters: requestOptions.queryParameters,
      options: Options(
        method: requestOptions.method,
        headers: requestOptions.headers,
      ),
    );
  }
}
```

**Explanation:**

- **Dio**: A powerful HTTP client for Dart, superior to the basic `http` package. Supports interceptors, request cancellation, file downloading, and global configuration.
- **Retrofit**: A type-safe HTTP client generator (inspired by Android's Retrofit). Uses code generation (`build_runner`) to create implementations from abstract classes with annotations.
- **`@JsonSerializable()`**: Part of `json_annotation` package. Generates `toJson()` and `fromJson()` methods using `build_runner`. Essential for type-safe JSON parsing.
- **Interceptors**: Middleware for HTTP requests/responses. Used for logging, authentication, error handling, and request modification.
- **Part/Part of**: The `part` directive links the main file to generated code (`api_service.g.dart`). The generated file contains the implementation of `_ApiService`.

### **State Management**

```dart
// Riverpod Example - Modern, compile-safe state management
import 'package:flutter_riverpod/flutter_riverpod.dart';

// Provider for simple immutable state
final counterProvider = StateProvider<int>((ref) => 0);
// StateProvider is for simple state that can change (primitives, simple objects)
// ref allows accessing other providers and managing lifecycle

// FutureProvider for async operations
final userProvider = FutureProvider<User>((ref) async {
  final apiService = ref.watch(apiServiceProvider);
  // ref.watch rebuilds when dependency changes
  return apiService.getCurrentUser();
});

// StreamProvider for real-time data
final messagesProvider = StreamProvider<List<Message>>((ref) {
  final repository = ref.watch(messageRepositoryProvider);
  return repository.messageStream();
});

// StateNotifier for complex state with business logic
class AuthState {
  final User? user;
  final bool isLoading;
  final String? error;
  
  AuthState({this.user, this.isLoading = false, this.error});
  
  AuthState copyWith({User? user, bool? isLoading, String? error}) {
    return AuthState(
      user: user ?? this.user,
      isLoading: isLoading ?? this.isLoading,
      error: error ?? this.error,
    );
  }
}

class AuthNotifier extends StateNotifier<AuthState> {
  final AuthRepository _repository;
  
  AuthNotifier(this._repository) : super(AuthState());
  
  Future<void> login(String email, String password) async {
    state = state.copyWith(isLoading: true, error: null);
    // Update state to loading
    
    try {
      final user = await _repository.login(email, password);
      state = state.copyWith(user: user, isLoading: false);
      // Update state with user data
    } catch (e) {
      state = state.copyWith(error: e.toString(), isLoading: false);
      // Update state with error
    }
  }
  
  void logout() {
    _repository.logout();
    state = AuthState();
    // Reset to initial state
  }
}

final authNotifierProvider = StateNotifierProvider<AuthNotifier, AuthState>((ref) {
  final repo = ref.watch(authRepositoryProvider);
  return AuthNotifier(repo);
});
```

**Explanation:**

- **Riverpod**: The successor to Provider, offering compile-time safety (no BuildContext needed), testability, and scoped state management.
- **Provider types**:
  - `Provider`: For immutable, computed values
  - `StateProvider`: For simple mutable state (int, String, etc.)
  - `FutureProvider`: For async operations (auto-handles loading/error states)
  - `StreamProvider`: For listening to Dart Streams
  - `StateNotifierProvider`: For complex state with business logic (BLoC alternative)
- **ref.watch**: Subscribes to a provider. When the provider updates, the widget rebuilds.
- **ref.read**: One-time read of provider value (use in callbacks, not build methods).
- **StateNotifier**: A class that holds state and exposes methods to modify it. Immutable state pattern (always create new state objects).

### **Local Storage**

```dart
import 'package:hive/hive.dart';
import 'package:hive_flutter/hive_flutter.dart';

// Type adapter for custom objects (code-generated)
@HiveType(typeId: 0)
class Todo {
  @HiveField(0)
  final String id;
  
  @HiveField(1)
  final String title;
  
  @HiveField(2)
  final bool isCompleted;
  
  @HiveField(3)
  final DateTime createdAt;
  
  Todo({
    required this.id,
    required this.title,
    this.isCompleted = false,
    required this.createdAt,
  });
}

class LocalStorageService {
  static const String _todoBoxName = 'todos';
  static Box<Todo>? _todoBox;
  
  // Initialize Hive
  static Future<void> init() async {
    await Hive.initFlutter();
    // Initialize Hive for Flutter (uses app document directory)
    
    Hive.registerAdapter(TodoAdapter());
    // Register generated adapter for Todo class
    
    _todoBox = await Hive.openBox<Todo>(_todoBoxName);
    // Open box (like a table in SQL)
  }
  
  // CRUD Operations
  static Future<void> addTodo(Todo todo) async {
    await _todoBox?.put(todo.id, todo);
    // Store by key (id)
  }
  
  static List<Todo> getAllTodos() {
    return _todoBox?.values.toList() ?? [];
    // Get all values in box
  }
  
  static Todo? getTodo(String id) {
    return _todoBox?.get(id);
    // Get by key
  }
  
  static Future<void> updateTodo(Todo todo) async {
    await _todoBox?.put(todo.id, todo);
    // Put overwrites existing key
  }
  
  static Future<void> deleteTodo(String id) async {
    await _todoBox?.delete(id);
  }
  
  // Watch for changes (reactive)
  static Stream<BoxEvent> watchTodos() {
    return _todoBox?.watch() ?? Stream.empty();
    // Returns stream of changes for reactive UI updates
  }
  
  static Future<void> clearAll() async {
    await _todoBox?.clear();
  }
}

// Secure storage for sensitive data
import 'package:flutter_secure_storage/flutter_secure_storage.dart';

class SecureStorage {
  static const _storage = FlutterSecureStorage(
    aOptions: AndroidOptions(
      encryptedSharedPreferences: true,
      // Use EncryptedSharedPreferences on Android
      keyCipherAlgorithm: KeyCipherAlgorithm.RSA_ECB_PKCS1Padding,
      storageCipherAlgorithm: StorageCipherAlgorithm.AES_GCM_NoPadding,
    ),
    iOptions: IOSOptions(
      accountName: 'flutter_secure_storage',
      accessibility: KeychainAccessibility.first_unlock_this_device,
      // iOS keychain accessibility
    ),
  );
  
  static Future<void> writeToken(String token) async {
    await _storage.write(key: 'auth_token', value: token);
    // Encrypted using platform keystore (Keychain/Keystore)
  }
  
  static Future<String?> readToken() async {
    return await _storage.read(key: 'auth_token');
  }
  
  static Future<void> deleteToken() async {
    await _storage.delete(key: 'auth_token');
  }
}
```

**Explanation:**

- **Hive**: A lightweight, fast NoSQL database written in pure Dart. Uses binary storage (not SQL). Best for structured local data caching.
- **TypeAdapters**: Hive stores binary data. `@HiveType` and `@HiveField` annotations generate adapters to convert objects to binary. Run `flutter pub run build_runner build` to generate `TodoAdapter`.
- **Box**: Hive's storage unit (equivalent to a table). Open boxes before use and close them when done.
- **flutter_secure_storage**: Stores data in platform-specific secure storage (iOS Keychain, Android Keystore). Use for tokens, passwords, API keys.
- **Reactive storage**: `box.watch()` returns a Stream of changes, enabling reactive UI updates without polling.

---

## **50.3 Evaluating Package Quality**

When selecting packages for production, rigorous evaluation is essential to avoid technical debt and security risks.

### **Pub.dev Scoring Criteria**

Pub.dev automatically scores packages (0-140 points) based on:

```dart
// Example of a high-quality package structure
// (This is how you should structure your own packages)

/*
package_name/
├── lib/
│   ├── package_name.dart          # Main export file
│   ├── src/
│   │   ├── core/
│   │   │   ├── models/
│   │   │   │   ├── user.dart
│   │   │   │   └── user.g.dart    # Generated code
│   │   │   ├── services/
│   │   │   │   └── api_service.dart
│   │   │   └── utils/
│   │   │       └── extensions.dart
│   │   └── package_name_base.dart # Private implementation
│   └── src.dart                   # Export internal parts (optional)
├── test/
│   ├── unit/
│   │   └── models_test.dart
│   ├── integration/
│   │   └── api_test.dart
│   └── mocks/
│       └── mock_data.dart
├── example/
│   └── lib/
│       └── main.dart              # Working example
├── CHANGELOG.md                   # Version history
├── LICENSE                        # BSD/MIT/Apache 2.0 recommended
├── README.md                      # Documentation with badges
└── pubspec.yaml
*/

// Evaluation Checklist Implementation
class PackageEvaluator {
  final String packageName;
  
  PackageEvaluator(this.packageName);
  
  Future<EvaluationResult> evaluate() async {
    // 1. Check popularity (pub.dev metrics)
    // Look for:
    // - Likes (community approval)
    // - Pub Points (code quality, documentation)
    // - Popularity (usage percentage)
    
    // 2. Verify maintenance health
    final healthChecks = await _checkMaintenance();
    
    // 3. Analyze dependencies
    final dependencyTree = await _analyzeDependencies();
    
    return EvaluationResult(
      isProductionReady: healthChecks.isUpToDate && 
                        healthChecks.hasTests &&
                        dependencyTree.hasNoVulnerabilities,
      score: _calculateScore(),
      recommendations: _generateRecommendations(),
    );
  }
  
  MaintenanceHealth _checkMaintenance() {
    return MaintenanceHealth(
      // Last commit within 6 months
      isRecentlyUpdated: true,
      // Responds to issues
      isMaintainerActive: true,
      // Null safety support (Dart 3 compatible)
      supportsNullSafety: true,
      // Has stable version (not 0.x.x)
      isStable: true,
      // Documentation exists
      hasDocumentation: true,
    );
  }
  
  DependencyAnalysis _analyzeDependencies() {
    // Check for:
    // - Transitive dependency conflicts
    // - Outdated packages
    // - Known vulnerabilities (via GitHub Advisory Database)
    return DependencyAnalysis(
      directDependencies: [],
      transitiveDependencies: [],
      vulnerabilities: [],
    );
  }
}
```

**Explanation:**

- **Pub Points (0-140)**:
  - **Follows Dart file conventions (20 points)**: Proper `lib/` structure, `README.md`, `CHANGELOG.md`, `LICENSE`.
  - **Provides documentation (20 points)**: Public API has dartdoc comments (`///`).
  - **Supports multiple platforms (20 points)**: iOS, Android, Web, macOS, Windows, Linux.
  - **Pass static analysis (30 points)**: No `dart analyze` errors or warnings.
  - **Up-to-date dependencies (20 points)**: Dependencies are current and resolve without conflicts.
  - **Supports latest SDKs (20 points)**: Compatible with latest stable Dart/Flutter.
  - **Bonus points (10 points)**: Has working `example/`, uses null safety.
- **Maintenance indicators**:
  - **Commit frequency**: Active development within last 3-6 months.
  - **Issue response**: Maintainer responds to critical issues within weeks.
  - **Breaking changes**: Handles versioning responsibly (semver).
  - **Null safety**: Dart 3 compatibility (no deprecated APIs).

### **Security and License Compliance**

```dart
// License validation utility
class LicenseValidator {
  static const approvedLicenses = [
    'MIT',
    'BSD-2-Clause',
    'BSD-3-Clause',
    'Apache-2.0',
    'WTFPL',
  ];
  
  static const copyleftLicenses = [
    'GPL-2.0',
    'GPL-3.0',
    'LGPL-2.1',
    'LGPL-3.0',
    'AGPL-3.0',
  ];
  
  static bool isApproved(String license) {
    return approvedLicenses.contains(license);
  }
  
  static bool isCopyleft(String license) {
    // Copyleft licenses may require you to open-source your app
    return copyleftLicenses.contains(license);
  }
  
  static void validatePackage(String name, String license) {
    if (isCopyleft(license)) {
      throw LicenseException(
        'Package $name uses copyleft license $license. '
        'This may require you to open-source your application.'
      );
    }
    
    if (!isApproved(license)) {
      print('Warning: Package $name uses unreviewed license: $license');
    }
  }
}

// Dependency security scanning
class SecurityScanner {
  static Future<List<Vulnerability>> scanDependencies() async {
    // Use tools like:
    // 1. `flutter pub audit` (experimental)
    // 2. GitHub Dependabot
    // 3. Snyk (snyk.io)
    // 4. OWASP Dependency-Check
    
    final vulnerabilities = <Vulnerability>[];
    
    // Check for known vulnerabilities in:
    // - Direct dependencies (your pubspec.yaml)
    // - Transitive dependencies (dependencies of dependencies)
    
    // Common vulnerabilities to check:
    // - Hardcoded API keys in packages
    // - HTTP instead of HTTPS in network packages
    // - Path traversal in file system packages
    // - Prototype pollution in JSON parsing
    
    return vulnerabilities;
  }
}
```

**Explanation:**

- **License types**:
  - **Permissive** (MIT, BSD, Apache 2.0): Allow commercial use, modification, and private use. Safe for proprietary apps.
  - **Copyleft** (GPL, LGPL): Require derivative works to be open-source under the same license. Generally avoid in proprietary commercial apps.
  - **Proprietary**: Custom licenses requiring legal review.
- **Security scanning**:
  - **Supply chain attacks**: Malicious packages mimicking popular ones (typosquatting like `dioo` instead of `dio`).
  - **Vulnerabilities**: Use `flutter pub audit` (when available) or third-party tools to scan for CVEs (Common Vulnerabilities and Exposures).
  - **Code review**: For critical packages, review the source code on GitHub before using.

---

## **50.4 Creating and Publishing Packages**

Creating reusable packages is a hallmark of senior Flutter developers. This section covers enterprise-grade package development.

### **Package Structure and Architecture**

```dart
// lib/my_package.dart - Main entry point
// Only export what you want to be public API

// Export specific files
export 'src/core/models.dart';
export 'src/core/api_client.dart';
export 'src/widgets/custom_button.dart';

// Hide internal implementation details
// Do NOT export: src/internal/helper.dart

// src/core/models.dart
import 'package:freezed_annotation/freezed_annotation.dart';

part 'models.freezed.dart';
part 'models.g.dart';

// Using Freezed for immutable data classes
@freezed
class ApiResponse<T> with _$ApiResponse<T> {
  const factory ApiResponse({
    required bool success,
    required T data,
    String? errorMessage,
    required DateTime timestamp,
  }) = _ApiResponse<T>;
  
  factory ApiResponse.fromJson(
    Map<String, dynamic> json,
    T Function(Object?) fromJsonT,
  ) =>
      _$ApiResponseFromJson<T>(json, fromJsonT);
}

// src/core/api_client.dart
abstract class ApiClient {
  Future<ApiResponse<T>> get<T>(
    String path, {
    required T Function(Map<String, dynamic>) parser,
    Map<String, dynamic>? queryParameters,
  });
  
  Future<ApiResponse<T>> post<T>(
    String path, {
    required Map<String, dynamic> body,
    required T Function(Map<String, dynamic>) parser,
  });
}

// Implementation hidden in src/internal/
// src/internal/dio_client.dart
import 'package:dio/dio.dart';

class DioApiClient implements ApiClient {
  final Dio _dio;
  
  DioApiClient(this._dio);
  
  @override
  Future<ApiResponse<T>> get<T>(
    String path, {
    required T Function(Map<String, dynamic>) parser,
    Map<String, dynamic>? queryParameters,
  }) async {
    try {
      final response = await _dio.get(
        path,
        queryParameters: queryParameters,
      );
      
      return ApiResponse(
        success: true,
        data: parser(response.data),
        timestamp: DateTime.now(),
      );
    } on DioException catch (e) {
      return ApiResponse(
        success: false,
        data: null as T, // Handle appropriately in production
        errorMessage: e.message,
        timestamp: DateTime.now(),
      );
    }
  }
  
  // ... post implementation
}

// src/widgets/custom_button.dart
import 'package:flutter/material.dart';

/// A customizable button following Material Design 3 guidelines.
/// 
/// Example usage:
/// ```dart
/// CustomButton(
///   onPressed: () => print('Clicked'),
///   child: Text('Submit'),
///   variant: ButtonVariant.primary,
/// )
/// ```
class CustomButton extends StatelessWidget {
  final VoidCallback? onPressed;
  final Widget child;
  final ButtonVariant variant;
  final bool isLoading;
  
  const CustomButton({
    super.key,
    required this.onPressed,
    required this.child,
    this.variant = ButtonVariant.primary,
    this.isLoading = false,
  });
  
  @override
  Widget build(BuildContext context) {
    return ElevatedButton(
      onPressed: isLoading ? null : onPressed,
      style: _getStyle(context),
      child: isLoading 
        ? const CircularProgressIndicator()
        : child,
    );
  }
  
  ButtonStyle _getStyle(BuildContext context) {
    return switch (variant) {
      ButtonVariant.primary => ElevatedButton.styleFrom(
          backgroundColor: Theme.of(context).colorScheme.primary,
        ),
      ButtonVariant.secondary => ElevatedButton.styleFrom(
          backgroundColor: Theme.of(context).colorScheme.secondary,
        ),
      ButtonVariant.danger => ElevatedButton.styleFrom(
          backgroundColor: Colors.red,
        ),
    };
  }
}

enum ButtonVariant { primary, secondary, danger }
```

**Explanation:**

- **Library structure**:
  - `lib/my_package.dart`: The main export file. Only expose public API here.
  - `lib/src/`: Internal implementation. Files here are private by convention (don't import from other packages' `src/` directories).
  - `lib/src/core/`: Core business logic.
  - `lib/src/widgets/`: Flutter widgets (if package is UI-related).
- **Documentation**:
  - `///` Dartdoc comments generate API documentation on pub.dev.
  - Include code examples in dartdoc using triple backticks.
  - Use `@param`, `@returns`, `@throws` for clarity.
- **Architecture**:
  - Hide implementation details in `src/internal/` or `src/impl/`.
  - Expose abstract classes/interfaces in public API.
  - Use dependency injection to allow mocking in tests.
- **Code generation**: Use `build_runner` for generating serialization code, but commit generated files only if they don't require the user to run build_runner (library packages should commit `.g.dart` files).

### **Testing Packages**

```dart
// test/api_client_test.dart
import 'package:mocktail/mocktail.dart';
import 'package:test/test.dart';
import 'package:my_package/my_package.dart';

// Mock classes
class MockDio extends Mock implements Dio {}
class MockResponse extends Mock implements Response<Map<String, dynamic>> {}

void main() {
  late MockDio mockDio;
  late ApiClient apiClient;
  
  setUp(() {
    mockDio = MockDio();
    apiClient = DioApiClient(mockDio);
  });
  
  group('ApiClient', () {
    test('get returns successful response', () async {
      // Arrange
      final mockResponse = MockResponse();
      when(() => mockResponse.data).thenReturn({'id': '1', 'name': 'Test'});
      
      when(() => mockDio.get(
        any(),
        queryParameters: any(named: 'queryParameters'),
      )).thenAnswer((_) async => mockResponse);
      
      // Act
      final result = await apiClient.get<Map<String, dynamic>>(
        '/users',
        parser: (json) => json,
      );
      
      // Assert
      expect(result.success, isTrue);
      expect(result.data, equals({'id': '1', 'name': 'Test'}));
      verify(() => mockDio.get('/users', queryParameters: null)).called(1);
    });
    
    test('get handles network errors', () async {
      // Arrange
      when(() => mockDio.get(any(), queryParameters: any(named: 'queryParameters')))
          .thenThrow(DioException(
            requestOptions: RequestOptions(path: '/users'),
            message: 'Network error',
          ));
      
      // Act
      final result = await apiClient.get<Map<String, dynamic>>(
        '/users',
        parser: (json) => json,
      );
      
      // Assert
      expect(result.success, isFalse);
      expect(result.errorMessage, equals('Network error'));
    });
  });
  
  group('Widget Tests', () {
    testWidgets('CustomButton displays child text', (tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: Scaffold(
            body: CustomButton(
              onPressed: () {},
              child: Text('Click Me'),
            ),
          ),
        ),
      );
      
      expect(find.text('Click Me'), findsOneWidget);
    });
    
    testWidgets('CustomButton shows loading indicator when isLoading is true', 
        (tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: Scaffold(
            body: CustomButton(
              onPressed: () {},
              isLoading: true,
              child: Text('Click Me'),
            ),
          ),
        ),
      );
      
      expect(find.byType(CircularProgressIndicator), findsOneWidget);
      expect(find.text('Click Me'), findsNothing);
    });
  });
}
```

**Explanation:**

- **Unit tests**: Test business logic in isolation. Use `mocktail` or `mockito` to mock dependencies.
- **Widget tests**: Test UI components using `testWidgets`. Pump the widget tree and interact with it using `tester.tap()`, `tester.enterText()`, etc.
- **Integration tests**: For packages, integration tests verify the package works with real dependencies (not mocked).
- **Coverage**: Aim for >80% code coverage. Run `flutter test --coverage` and use tools like `lcov` to generate HTML reports.

### **Publishing to pub.dev**

```yaml
# pubspec.yaml for publishing
name: my_awesome_package
description: >
  A comprehensive Flutter package that solves specific problems 
  with high performance and minimal dependencies.
# Description must be informative for pub.dev SEO

version: 1.0.0
homepage: https://github.com/username/my_awesome_package
repository: https://github.com/username/my_awesome_package
issue_tracker: https://github.com/username/my_awesome_package/issues
documentation: https://pub.dev/documentation/my_awesome_package/latest/

environment:
  sdk: '>=3.0.0 <4.0.0'
  flutter: ">=3.10.0"

dependencies:
  flutter:
    sdk: flutter
  # Minimize dependencies to reduce transitive dependency conflicts

dev_dependencies:
  flutter_test:
    sdk: flutter
  flutter_lints: ^3.0.0  # Official lint rules

flutter:
  # If package includes assets:
  # assets:
  #   - images/
```

```dart
// Pre-publish checklist in code comments
/*
PRE-PUBLISH CHECKLIST:

1. Version bump
   - Update version in pubspec.yaml following semver
   - Update CHANGELOG.md with version date and changes
   
2. Quality checks
   - flutter analyze (no errors/warnings)
   - flutter test (all passing)
   - dart format lib/ test/ (consistent formatting)
   
3. Documentation
   - All public APIs have dartdoc comments (///)
   - README.md has installation instructions and usage examples
   - example/ directory has working demo
   
4. Metadata
   - Add topics to pubspec.yaml (Dart 3.0+):
     topics:
       - networking
       - state-management
   - Include screenshots if UI package:
     screenshots:
       - description: 'Main interface'
         path: screenshots/main.png
   
5. Publish dry-run
   - flutter pub publish --dry-run
   - Verify no warnings about README, LICENSE, or CHANGELOG
   
6. Git tagging
   - git tag v1.0.0
   - git push origin v1.0.0
   
7. Publish
   - flutter pub publish
   - Verify package appears on pub.dev
*/
```

**Explanation:**

- **Metadata fields**:
  - `homepage`: Project website or GitHub repo.
  - `repository`: Source code location.
  - `issue_tracker`: Link to GitHub issues.
  - `documentation`: Link to generated docs.
- **CHANGELOG.md format**:
  ```markdown
  ## 1.0.0
  - Initial stable release.
  - Added support for Dart 3.
  
  ## 0.9.0
  - BREAKING: Changed ApiClient constructor signature.
  - Added support for Web platform.
  ```
- **Publishing process**:
  1. Verify `pubspec.yaml` metadata.
  2. Run `flutter pub publish --dry-run` to check for issues.
  3. Fix any warnings (missing documentation, invalid version constraints).
  4. Tag the release in Git (`git tag v1.0.0`).
  5. Run `flutter pub publish`.
  6. Verify on pub.dev that screenshots, documentation, and README render correctly.

---

## **50.5 Version Constraint Management**

Managing dependency versions is critical for long-term maintenance and security.

### **Semantic Versioning Strategy**

```dart
// pubspec.yaml strategies for different scenarios

// For APPLICATIONS (apps):
dependencies:
  # Use caret (^) to allow non-breaking updates
  dio: ^5.0.0
  # Allows 5.0.0 through 5.999.999, but not 6.0.0
  
  # Pin critical dependencies to exact versions if they're unstable
  payment_gateway: 2.1.0
  
  # Use environment variables for sensitive or environment-specific versions
  internal_sdk: 
    hosted:
      url: ${INTERNAL_PUB_URL}

// For LIBRARIES (packages):
dependencies:
  # Use wide constraints to maximize compatibility
  http: '>=0.13.0 <2.0.0'
  # Allows consumers to use any compatible version
  
  # For Flutter SDK dependencies, use compatible ranges
  flutter:
    sdk: flutter
    
dependency_overrides:
  # Use sparingly and temporarily
  # transmogrify: ^3.2.0
```

**Explanation:**

- **Applications vs. Libraries**:
  - **Applications**: Use `^` constraints and commit `pubspec.lock`. This ensures reproducible builds but requires active maintenance to update dependencies.
  - **Libraries**: Use wide ranges (`>=1.0.0 <3.0.0`) to avoid version conflicts for consumers. Don't commit `pubspec.lock` in library repos.
- **Breaking changes**: When a dependency releases a major version (e.g., `2.0.0`):
  - Review the migration guide.
  - Update constraint in `pubspec.yaml`: `^2.0.0`.
  - Update import statements if package structure changed.
  - Run full regression tests.
- **Lock file strategy**:
  - **CI/CD**: Use `flutter pub get` (respects lock file) for reproducible builds.
  - **Updates**: Use `flutter pub upgrade` weekly to get patches, monthly for minor updates.

### **Dependency Conflict Resolution**

```dart
// Handling version conflicts example

/*
CONFLICT SCENARIO:
Package A depends on http: ^0.13.0
Package B depends on http: ^1.0.0
These are incompatible because http 1.0.0 is a major version change from 0.13.0
*/

// Resolution strategies:

// 1. Dependency overrides (temporary fix)
dependency_overrides:
  http: ^1.0.0
  # Force all packages to use http 1.0.0
  # Risk: Package A might break if not compatible with http 1.0.0

// 2. Fork and patch (if Package A is unmaintained)
dependencies:
  package_a:
    git:
      url: https://github.com/yourorg/package_a_fork.git
      ref: http-1.0-compat

// 3. Use dependency constraints to find compatible versions
// Sometimes using an older version of Package B that supports http 0.13.0
dependencies:
  package_b: ^2.0.0  # Instead of ^3.0.0 which requires http 1.0.0

// 4. Contact maintainers or contribute PRs to update dependencies
```

**Explanation:**

- **Version solving**: Pub uses constraint satisfaction. If two packages require incompatible versions of a third package, resolution fails.
- **Conflict resolution hierarchy**:
  1. Check if a middle version satisfies both (rare for major version conflicts).
  2. Use `dependency_overrides` to force a version (test thoroughly).
  3. Fork the unmaintained package and update it yourself.
  4. Wait for maintainers to update (file an issue).
- **Transitive dependencies**: Use `flutter pub deps` to visualize the tree. Look for packages depending on old versions.
- **SDK constraints**: Ensure your `environment` SDK constraints allow the dependency versions you need.

---

## **Chapter Summary**

In this chapter, we covered the Dart package ecosystem essential for professional Flutter development:

### **Key Takeaways:**

1. **Pubspec.yaml mastery**: Understanding version constraints (`^`, ranges, exact), environment SDK requirements, and dependency types (direct vs. dev vs. transitive).
2. **Pub commands**: `get` for installation, `upgrade` for updates, `outdated` for maintenance, `deps` for analysis, and `publish` for distribution.
3. **Essential packages**:
   - **Networking**: Dio + Retrofit for type-safe HTTP with interceptors
   - **State Management**: Riverpod for compile-safe, testable state
   - **Storage**: Hive for fast NoSQL, flutter_secure_storage for sensitive data
4. **Package evaluation**: Check pub points (0-140), maintenance health (commits, issues), license compatibility (avoid copyleft for proprietary apps), and security (scan for vulnerabilities).
5. **Package creation**:
   - Structure: Public API in `lib/`, implementation in `lib/src/`
   - Documentation: Dartdoc comments, working example, comprehensive README
   - Testing: Unit tests with mocking, widget tests for UI, >80% coverage
   - Publishing: Proper metadata, CHANGELOG maintenance, Git tagging
6. **Version management**: Apps use `pubspec.lock` for reproducibility; libraries use wide constraints for compatibility. Handle conflicts via overrides or forking.
7. **Security**: Audit dependencies for CVEs, validate licenses (MIT/Apache preferred), avoid typosquatting attacks.

### **Industry Standards Checklist:**

- [ ] Pin critical dependencies; use caret for others
- [ ] Audit packages quarterly for security updates
- [ ] Maintain CHANGELOG.md and semantic versioning
- [ ] Include comprehensive dartdoc comments
- [ ] Support all target platforms (iOS, Android, Web, Desktop)
- [ ] Include working example app
- [ ] Use MIT/BSD/Apache 2.0 licenses for open source
- [ ] Automate testing in CI/CD before publishing

### **Next Steps:**

The next chapter (Chapter 51) will cover **Platform-Specific Configuration**, detailing:
- AndroidManifest.xml and Gradle configuration
- iOS Info.plist and Podfile management
- Platform-specific permission handling
- ProGuard/R8 rules for release builds
- Gradle and CocoaPods troubleshooting

---

**End of Chapter 50**

---

# **Next Chapter: Chapter 51 - Platform-Specific Configuration**

Chapter 51 will dive into the native platform configurations required for production Flutter applications, covering Android and iOS-specific setup, permission management, and build optimization for each platform.

